# Importing Libraries

In [1]:
import os
import gspread
from google.oauth2 import service_account
from googleapiclient.discovery import build

In [2]:
def authenticate_drive(credentials_path):
    """Authenticate and return the Google Drive API service."""
    credentials = service_account.Credentials.from_service_account_file(
        credentials_path,
        scopes=['https://www.googleapis.com/auth/drive']
    )
    drive_service = build('drive', 'v3', credentials=credentials)
    return drive_service

# Code To Read and Send Files

In [ ]:
import pandas as pd

data = pd.read_excel("file.xlsx")

In [3]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os
import pickle

# Define the scopes
SCOPES = ['https://www.googleapis.com/auth/drive']

def get_gdrive_service():
  creds = None
  if os.path.exists('token.pickle'):
      with open('token.pickle', 'rb') as token:
          creds = pickle.load(token)
  if not creds or not creds.valid:
      if creds and creds.expired and creds.refresh_token:
          creds.refresh(Request())
      else:
          flow = InstalledAppFlow.from_client_secrets_file(
              "client_secret_158237910863-j94jdkja01eh0r5hi7qqm2evhcb2ntgb.apps.googleusercontent.com.json", SCOPES)
          creds = flow.run_local_server(port=0)
      with open('token.pickle', 'wb') as token:
          pickle.dump(creds, token)
  return build('drive', 'v3', credentials=creds)


In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.utils import make_msgid

message = """
The message to write


 """

def send_email(to_address, subject, body, signature_url):
    """
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = "Myemail@email.com"
    msg['To'] = to_address

    body_html = f"{body}<br><img src='{signature_url}'>"
    msg.attach(MIMEText(body_html, 'html'))

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login("Myemail@email.com", "pbel qjge zwcw plsf") # Use the application-specific password here
    server.send_message(msg)
    server.quit()
 """
    msg = f"Subject: {subject}\n\n{body}"
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login("Myemail@email.com", "pbel qjge zwcw plsf")
    server.sendmail("Myemail@email.com", to_address, msg)
    server.quit()


def search(service, query):
   result = []
   page_token = None
   while True:
       response = service.files().list(q=query, spaces='drive', fields='nextPageToken, files(id, name)', pageToken=page_token).execute()
       for file in response.get('files', []):
           result.append((file.get('id'), file.get('name')))
       page_token = response.get('nextPageToken', None)
       if page_token is None:
           break
   return result

service = get_gdrive_service()
for index, row in data.iterrows():
 email = row['Email']
 file_name = row['file_name']
 file_id = search(service, f"name='{file_name}'")[0][0]
 search_results = search(service, f"name='{file_name}'")
 if search_results:
    file_id = search_results[0][0]
 else:
    print(f"No file found with the name '{file_name}'")
    continue
 file_metadata = {'copyRequiresWriterPermission': False}
 file_metadata = {
   'writersCanInvite': True
}
 service.files().update(fileId=file_id, body=file_metadata).execute()
 service.files().update(fileId=file_id, body=file_metadata).execute()
 service.permissions().create(
     body={
         'role':  'commenter',
         'type': 'user',
         'emailAddress': email,
         'emailMessage':message,
         'withLink': True
     },
     fileId=file_id
 ).execute()


 'signature_url = "https://drive.google.com/file/d/11/view?usp=drive_link"'
 signature = """
   
"""
 send_email(email, "Header", message,signature)

In [ ]:
def search(service, query):
   result = []
   page_token = None
   while True:
       response = service.files().list(q=query, spaces='drive', fields='nextPageToken, files(id, name)', pageToken=page_token).execute()
       for file in response.get('files', []):
           result.append((file.get('id'), file.get('name')))
       page_token = response.get('nextPageToken', None)
       if page_token is None:
           break
   return result

service = get_gdrive_service()
for index, row in data.iterrows():
 email = row['Email']
 file_name = row['file_name']
 email = row['Email']
 file_name = row['file_name']
 
 search_results = search(service, f"name='{file_name}'")
 if search_results:
    file_id = search_results[0][0]
    print(f"File found with the name '{file_name}'")
 else:
    print(f"No file found with the name '{file_name}'")
    continue

In [ ]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os.path
import pickle


# If modifying these SCOPES, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive']

def main():
   """Shows basic usage of the Drive v3 API.
   Prints the names and ids of the first 10 files the user has access to.
   """
   creds = None
   # The file token.pickle stores the user's access and refresh tokens, and is
   # created automatically when the authorization flow completes for the first
   # time.
   if os.path.exists('token.pickle'):
       with open('token.pickle', 'rb') as token:
           creds = pickle.load(token)
   # If there are no (valid) credentials available, let the user log in.
   if not creds or not creds.valid:
       if creds and creds.expired and creds.refresh_token:
           creds.refresh(Request())
       else:
           flow = InstalledAppFlow.from_client_secrets_file(
               'Token.json', SCOPES)
           creds = flow.run_local_server(port=0)
       # Save the credentials for the next run
       with open('token.pickle', 'wb') as token:
           pickle.dump(creds, token)

   service = build('drive', 'v3', credentials=creds)

   # Call the Drive v3 API
   results = service.files().list(
       pageSize=10, fields="nextPageToken, files(id, name)").execute()
   items = results.get('files', [])

   if not items:
       print('No files found.')
   else:
       print('Files:')
       for item in items:
           print(u'{0} ({1})'.format(item['name'], item['id']))
           comments = service.comments().list(fileId=item['id'], fields='comments').execute()
           if comments:
               print(f"File {item['name']} has comments.")
           else:
               print(f"File {item['name']} does not have comments.")

if __name__ == '__main__':
   main()
